In [1]:
!pip install Sastrawi

     -------------------------------------- 209.7/209.7 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install spacy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from gensim.models.nmf import Nmf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import plotly.graph_objs as go
from plotly.offline import iplot
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

c:\Users\zhafr\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [12]:
# Load the data
df = pd.read_excel(r'C:\Users\zhafr\OneDrive\Documents\Dataset\forestFires-dataset.xlsx')
df = df[['titles', 'articles']]
df.drop_duplicates(inplace=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1863 entries, 0 to 1862
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   titles    1863 non-null   object
 1   articles  1863 non-null   object
dtypes: object(2)
memory usage: 29.2+ KB


In [13]:
# Define function to clean the text
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    return text

# Apply text cleaning
df['cleaned_articles'] = df['articles'].apply(clean_text)

In [14]:
# Vectorize the text using TfidfVectorizer
stop_words = ['dan', 'the', 'di', 'dari', 'ke', '\n', 'ago', 'days', 'doyoung', 
              'hee', 'julian', 'band', 'connelly', 'bts', 'jennifer', 'won', 'picchu', 
              'machu', 'suga', 'film', 'sesar', 'yellen',
              'prasmul', 'maverick', 'labyrinth', 'memaklumkan', 'menceritakan', 
              'waterloo', 'dataset', 'fincapes', 'dibintangi', 'elsa',
              'klhk', 'kementerian', 'ha', 'restorasi', 'emisi', 'hektare', 'karbon', 'juta', 'siti', 'gambut']

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words=stop_words)
vectorized_text = vectorizer.fit_transform(df['cleaned_articles'])

In [15]:
# Define function to display the topics
def display_topics(model, feature_names_out, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)] = ['{}'.format(feature_names_out[i]) for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)] = ['{:.1f}'.format(topic[i]) for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [16]:
# Fit the NMF model
NUM_TOPICS = 10

nmf = NMF(n_components=NUM_TOPICS,max_iter=10, random_state=42)
nmf.fit(vectorized_text)

c:\Users\zhafr\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning:

Maximum number of iterations 10 reached. Increase it to improve convergence.



NMF(max_iter=10, n_components=10, random_state=42)

In [17]:
# Display the topics
no_top_words = 10
display_topics(nmf, vectorizer.get_feature_names_out(), no_top_words)

# Normalize the vectorized text
normalizer = Normalizer()
normalized_text = normalizer.fit_transform(vectorized_text)

In [18]:
# Reduce the dimensionality of the vectorized text
svd = TruncatedSVD(n_components=50, random_state=42)
tsvd_text = svd.fit_transform(normalized_text)
tsne = TSNE(random_state=42)
tsne_text = tsne.fit_transform(tsvd_text)


In [ ]:
# Create a scatter plot of the t-SNE reduced data
trace = go.Scattergl(
    x = tsne_text[:,0],
    y = tsne_text[:,1],
    mode = 'markers',
    marker = dict(
        color = df['articles'].astype('category').cat.codes,
        colorscale = 'Jet',
        line = dict(width = 1)
    ),
    text = df['articles'],
    hovertext = df['cleaned_articles'],
    hoverinfo = 'text' 
)

data = [trace]
layout = go.Layout(
    autosize=False,
    width=800,
    height=800
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)


In [ ]:
# Get the feature names from the TfidfVectorizer object
feature_names = vectorizer.get_feature_names_out()

# Print top 10 words per topic
for i, topic in enumerate(nmf.components_):
    print(f"Top 10 words for topic #{i}:")
    print([feature_names[index] for index in topic.argsort()[-10:]])
    print('\n')

#Evaluasi Error

In [ ]:
reconstruction_error = nmf.reconstruction_err_

# print reconstruction error
print("Reconstruction error:", reconstruction_error)

In [ ]:
from sklearn.metrics import silhouette_score

# membuat model NMF


# assign cluster labels untuk data
cluster_labels = nmf.transform(vectorized_text).argmax(axis=1)

# hitung silhouette score
silhouette_avg = silhouette_score(vectorized_text, cluster_labels)

# print silhouette score
print("Silhouette score:", silhouette_avg)

In [ ]:
from gensim.parsing.preprocessing import (
    strip_non_alphanum,
    split_alphanum,
    strip_short,
    strip_numeric
)
import gensim

from gensim.models import Phrases

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

x = df['cleaned_articles'].values.tolist()
# membuat kamus kata
docs = list(sent_to_words(x))
dictionary = Dictionary(docs)
# membuat representasi bag-of-words dari dokumen
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:


# membuat model NMF
nmf_model = Nmf(corpus=corpus, num_topics=10)

# hitung coherence score
coherence_model = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# print coherence score
print("Coherence score:", coherence_score)


In [ ]:
from wordcloud import WordCloud
# Generate a word cloud image for given topic
def draw_word_cloud(index):
  imp_words_topic=""
  comp=nmf.components_[index]
  vocab_comp = zip(feature_names, comp)
  sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:50]
  for word in sorted_words:
    imp_words_topic=imp_words_topic+" "+word[0]

  wordcloud = WordCloud(width=600, height=400).generate(imp_words_topic)
  plt.figure( figsize=(5,5))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout()
  plt.show()

In [ ]:
draw_word_cloud(0)

In [ ]:
draw_word_cloud(1)